In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
5,Now we have proof that kids are paying the pri...,https://www.cnn.com/2020/12/02/politics/what-m...,"Wed, 02 Dec 2020 05:04:00 GMT",2020-12-02 05:04:00+00:00
7,San Diego County Crosses 1000 COVID-19 Deaths ...,https://www.kpbs.org/news/2020/dec/01/san-dieg...,"Wed, 02 Dec 2020 01:32:19 GMT",2020-12-02 01:32:19+00:00
3,Meet the scientists investigating the origins ...,https://www.nature.com/articles/d41586-020-034...,"Wed, 02 Dec 2020 00:37:36 GMT",2020-12-02 00:37:36+00:00
0,"Coronavirus (COVID-19) Update: December 1, 202...",https://www.fda.gov/news-events/press-announce...,"Wed, 02 Dec 2020 00:12:53 GMT",2020-12-02 00:12:53+00:00
9,"COVID-19 Scan for Dec 01, 2020 | CIDRAP CIDRAP",https://www.cidrap.umn.edu/news-perspective/20...,"Tue, 01 Dec 2020 22:36:13 GMT",2020-12-01 22:36:13+00:00
4,Governor Cuomo Updates New Yorkers on State's ...,https://www.governor.ny.gov/news/governor-cuom...,"Tue, 01 Dec 2020 19:33:02 GMT",2020-12-01 19:33:02+00:00
1,Federal Advisory Committee Votes On COVID-19 V...,https://www.npr.org/sections/health-shots/2020...,"Tue, 01 Dec 2020 18:37:42 GMT",2020-12-01 18:37:42+00:00
6,Apple PPE donation helps Zambia in the fight a...,https://www.apple.com/newsroom/2020/12/apple-p...,"Tue, 01 Dec 2020 08:04:58 GMT",2020-12-01 08:04:58+00:00
2,Apple expands partnership with RED to combat H...,https://www.apple.com/newsroom/2020/12/apple-e...,"Tue, 01 Dec 2020 08:00:42 GMT",2020-12-01 08:00:42+00:00
8,COVID-19 Daily Update 11-30-2020 West Virgini...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Mon, 30 Nov 2020 15:00:11 GMT",2020-11-30 15:00:11+00:00
